
# 🥇 Etapa **ORO** · Sesiones, Embudo y Reporte (Web Logs)

Este notebook implementa la fase **ORO** del pipeline **BRONCE → PLATA → ORO**:

- **Sesionización** con *timeout* (gap en minutos) y `session_id` estable.
- **Embudo por sesión** ( `/ → /productos → /carrito → /checkout` ) con soporte para **múltiples compras** por sesión.
- **Tablas agregadas**: sesiones, `users_stats`, **top 10 paths**, **uso de dispositivos**, **sesiones/día** y **funnel**.
- **Reporte Markdown** con los KPIs clave y tablas.

El código reimplementa `oro.py` y `report.py` de forma **autosuficiente** en el notebook (sin dependencias externas) y añade una **demo mínima**.


## 📦 Importaciones

In [ ]:

import hashlib
from types import SimpleNamespace as Namespace
import pandas as pd
from pathlib import Path

BASE_DIR = Path('/mnt/data')
BASE_DIR.mkdir(parents=True, exist_ok=True)
print("BASE_DIR:", BASE_DIR)


## 🔧 Utilidades de sesión

In [ ]:

def idx(lst, val):
    try:
        return lst.index(val)
    except ValueError:
        return None

def make_session_id(row) -> str:
    # Estable por (user_id, date, session_idx)
    base = f"{row.user_id}|{row.date}|{int(row.session_idx)}"
    return hashlib.sha1(base.encode()).hexdigest()[:16]


## 🛒 Detección de embudo por sesión

In [ ]:

def detect_session_funnel_with_counts(paths: list[str]) -> dict:
    """
    Devuelve flags de 'paso visto' (al menos una vez, en orden) y
    'purchases_in_session' (número de secuencias completas).
    Secuencia base: / -> /productos -> /carrito -> /checkout

    Nota: Para permitir múltiples compras en la misma sesión aunque no se vuelva
    a '/', reiniciamos el ciclo tras un /checkout y permitimos empezar el ciclo
    en /productos si ya hubo '/' en la sesión (más realista en ecommerce).
    """
    saw_root = ("/" in paths)

    i_root = idx(paths, "/")
    i_prod = idx(paths, "/productos")
    i_cart = idx(paths, "/carrito")
    i_chk  = idx(paths, "/checkout")

    saw_prod_after_root = (i_root is not None and i_prod is not None and i_prod > i_root)
    saw_cart_after_prod = (saw_prod_after_root and i_cart is not None and i_cart > i_prod)
    saw_chk_after_cart  = (saw_cart_after_prod and i_chk  is not None and i_chk  > i_cart)

    purchases = 0
    have_seen_root = False
    state = "start"

    for p in paths:
        if p == "/":
            have_seen_root = True
            state = "after_root"
            continue

        if state == "after_root":
            if p == "/productos":
                state = "prod"
            continue

        if state == "start" and have_seen_root:
            if p == "/productos":
                state = "prod"
            continue

        if state == "prod":
            if p == "/carrito":
                state = "cart"
            elif p == "/":
                state = "after_root"
            else:
                pass
            continue

        if state == "cart":
            if p == "/checkout":
                purchases += 1
                state = "start"
            elif p == "/":
                state = "after_root"
            else:
                pass
            continue

    return {
        "saw_root": saw_root,
        "saw_productos_after_root": saw_prod_after_root,
        "saw_carrito_after_productos": saw_cart_after_prod,
        "saw_checkout_after_carrito": saw_chk_after_cart,
        "purchases_in_session": purchases,
    }


## 🥇 Función principal: `build_gold(silver, session_timeout_min)`

In [ ]:

def build_gold(silver: pd.DataFrame, session_timeout_min: int = 30):
    """
    Devuelve:
      - events_oro: eventos con session_id
      - sessions: tabla de sesiones con flags embudo y métricas
      - users_stats: visitas por usuario, sesiones y compras
      - top_paths: top 10 páginas
      - device_usage: uso de dispositivos (global)
      - sessions_per_day: nº sesiones por día
      - funnel_table: embudo agregado con tasas
    """
    df = silver.sort_values(["user_id", "ts"]).copy()

    # Sesionizar (gap > timeout => nueva sesión)
    df["prev_ts"] = df.groupby("user_id")["ts"].shift()
    df["gap_min"] = (df["ts"] - df["prev_ts"]).dt.total_seconds() / 60.0
    df["is_new_session"] = df["prev_ts"].isna() | (df["gap_min"] > float(session_timeout_min))
    df["session_idx"] = df.groupby("user_id")["is_new_session"].cumsum()

    # id estable por (user_id, date, session_idx)
    df["session_id"] = df.apply(make_session_id, axis=1)

    # Flags de embudo por sesión
    paths_by_session = (
        df.sort_values("ts")
          .groupby("session_id")["path"].apply(list)
          .to_dict()
    )
    flags_rows = []
    for sid, plist in paths_by_session.items():
        flags = detect_session_funnel_with_counts(plist)
        flags["session_id"] = sid
        flags_rows.append(flags)
    session_flags = pd.DataFrame(flags_rows)

    # Tabla de sesiones con métricas
    sessions = (
        df.groupby("session_id")
          .agg(
              user_id=("user_id", "first"),
              date=("date", "first"),
              start_ts=("ts", "min"),
              end_ts=("ts", "max"),
              pageviews=("path", "count"),
              device_first=("device", "first"),
        )
        .reset_index()
        .merge(session_flags, on="session_id", how="left")
    )
    sessions["session_duration_sec"] = (sessions["end_ts"] - sessions["start_ts"]).dt.total_seconds().fillna(0)

    # Métricas por usuario
    users_sessions = sessions.groupby("user_id").agg(
        sessions=("session_id", "nunique"),
        purchases=("purchases_in_session", "sum"),
        avg_session_duration_sec=("session_duration_sec", "mean"),
    )
    users_events = df.groupby("user_id").size().rename("events")
    users_stats = (
        users_sessions.merge(users_events, on="user_id", how="left")
        .reset_index()
        .sort_values(["purchases", "sessions", "events"], ascending=[False, False, False])
    )

    # Top 10 paths
    top_paths = (
        df["path"].value_counts().rename_axis("path").reset_index(name="views").head(10)
    )

    # Uso de dispositivos (global)
    device_usage = (
        df["device"].value_counts(dropna=True).rename_axis("device").reset_index(name="events")
    )

    # Sesiones por día
    sessions_per_day = (
        sessions.groupby("date")["session_id"].nunique().rename("sessions").reset_index()
    )

    # Embudo agregado
    total_sessions = len(sessions)
    s_root = int(sessions["saw_root"].sum())
    s_prod = int(sessions["saw_productos_after_root"].sum())
    s_cart = int(sessions["saw_carrito_after_productos"].sum())
    s_chk  = int(sessions["saw_checkout_after_carrito"].sum())

    funnel_table = pd.DataFrame({
        "step": [
            "Sesiones", "→ con '/'", "→ luego '/productos'", "→ luego '/carrito'", "→ luego '/checkout' (compra)"
        ],
        "count": [total_sessions, s_root, s_prod, s_cart, s_chk],
    })

    def safe_div(a, b): return (a / b) if b else 0.0
    funnel_table["rate_step"] = [
        1.0,
        safe_div(s_root, total_sessions),
        safe_div(s_prod, s_root),
        safe_div(s_cart, s_prod),
        safe_div(s_chk,  s_cart),
    ]
    funnel_table["rate_overall"] = [
        1.0,
        safe_div(s_root, total_sessions),
        safe_div(s_prod, total_sessions),
        safe_div(s_cart, total_sessions),
        safe_div(s_chk,  total_sessions),
    ]

    # Limpieza columnas intermedias en eventos
    events_oro = df.drop(columns=["prev_ts"]).copy()

    return events_oro, sessions, users_stats, top_paths, device_usage, sessions_per_day, funnel_table


## 📝 Generador de reporte Markdown (autosuficiente)

In [ ]:

FILE_GOLD_NAME   = 'events_oro.parquet'
FILE_SILVER_NAME = 'events_plata.parquet'

def build_report_md(args: Namespace,
                    bronze_df: pd.DataFrame,
                    bad_df: pd.DataFrame,
                    silver: pd.DataFrame,
                    sessions: pd.DataFrame,
                    users_stats: pd.DataFrame,
                    top_paths: pd.DataFrame,
                    device_usage: pd.DataFrame,
                    sessions_per_day: pd.DataFrame,
                    funnel: pd.DataFrame) -> str:
    """Devuelve el texto Markdown del reporte final."""
    import datetime
    gen_ts = datetime.datetime.now(datetime.timezone.utc).isoformat()

    total_sessions = len(sessions)
    total_purchases = int(sessions.get("purchases_in_session", pd.Series(dtype=int)).sum()) if "purchases_in_session" in sessions.columns else 0
    uniq_users = int(silver["user_id"].nunique()) if not silver.empty else 0
    avg_pages_per_session = float(sessions["pageviews"].mean()) if "pageviews" in sessions.columns and not sessions.empty else 0.0
    avg_session_min = float(sessions["session_duration_sec"].mean() / 60.0) if "session_duration_sec" in sessions.columns and not sessions.empty else 0.0

    top_paths_md = top_paths.to_markdown(index=False) if (isinstance(top_paths, pd.DataFrame) and not top_paths.empty) else "_(sin datos)_"
    device_usage_md = device_usage.to_markdown(index=False) if (isinstance(device_usage, pd.DataFrame) and not device_usage.empty) else "_(sin datos)_"
    sessions_per_day_md = sessions_per_day.to_markdown(index=False) if (isinstance(sessions_per_day, pd.DataFrame) and not sessions_per_day.empty) else "_(sin datos)_"
    funnel_md = funnel.to_markdown(index=False) if (isinstance(funnel, pd.DataFrame) and not funnel.empty) else "_(sin datos)_"

    bronze_rows = len(bronze_df)
    silver_rows = len(silver)
    bad_json_rows = len(bad_df)

    report = (
        "# Reporte · Web Logs (BRONCE → PLATA → ORO)\n"
        f"**Día:** {args.day} · **Fuente:** events.ndjson · **Generado:** {gen_ts}\n\n"
        "## 1. Titular\n"
        f"Usuarios únicos {uniq_users}; sesiones {total_sessions}; compras {total_purchases}.\n\n"
        "## 2. KPIs\n"
        f"- **Usuarios únicos:** {uniq_users}\n"
        f"- **Sesiones:** {total_sessions}\n"
        f"- **Compras (checkouts):** {total_purchases}\n"
        f"- **Eventos (plata):** {silver_rows}\n"
        f"- **Páginas por sesión (media):** {avg_pages_per_session:.2f}\n"
        f"- **Duración media sesión (min):** {avg_session_min:.2f}\n\n"
        "## 3. Top 10 páginas\n"
        f"{top_paths_md}\n\n"
        "## 4. Uso de dispositivos (por eventos)\n"
        f"{device_usage_md}\n\n"
        "## 5. Sesiones por día\n"
        f"{sessions_per_day_md}\n\n"
        "## 6. Embudo por sesión\n"
        f"{funnel_md}\n\n"
        "## 7. Calidad y cobertura\n"
        f"- Filas BRONCE: {bronze_rows}\n"
        f"- Filas PLATA: {silver_rows}\n"
        f"- Líneas rotas (JSON) a cuarentena: {bad_json_rows}\n"
        f"- Diferencia BRONCE→PLATA (drops/dedupe/fuera de día): {bronze_rows - silver_rows}\n\n"
        f"- Porcentaje de cobertura PLATA/BRONCE:  {(silver_rows/(bronze_rows+bad_json_rows)*100.0) if bronze_rows > 0 else 0.0:.2f}%\n\n"
        "## 8. Persistencia\n"
        f"- Parquet PLATA: `{args.silver}/{FILE_SILVER_NAME}`\n"
        f"- Parquet ORO: `{args.gold}/{FILE_GOLD_NAME}`\n"
        f"- Reporte: `{args.report}/{args.day}-reporte.md`\n"
    )
    return report



## 🧪 Demostración mínima (de PLATA → ORO)

Creamos un **DataFrame tipo PLATA** de ejemplo y ejecutamos `build_gold`.  
Luego generamos el **reporte Markdown** y lo guardamos en disco.


In [ ]:

# Silver (PLATA) de ejemplo
silver = pd.DataFrame([
    # user u1 hace una compra y luego otra en la misma sesión
    {"ts": pd.Timestamp("2025-11-08T10:00:00Z"), "user_id": "u1", "path": "/",          "referrer": "direct",   "device": "mobile",  "date": "2025-11-08"},
    {"ts": pd.Timestamp("2025-11-08T10:02:00Z"), "user_id": "u1", "path": "/productos",  "referrer": "direct",   "device": "mobile",  "date": "2025-11-08"},
    {"ts": pd.Timestamp("2025-11-08T10:03:00Z"), "user_id": "u1", "path": "/carrito",    "referrer": "direct",   "device": "mobile",  "date": "2025-11-08"},
    {"ts": pd.Timestamp("2025-11-08T10:04:00Z"), "user_id": "u1", "path": "/checkout",   "referrer": "direct",   "device": "mobile",  "date": "2025-11-08"},
    {"ts": pd.Timestamp("2025-11-08T10:06:00Z"), "user_id": "u1", "path": "/carrito",    "referrer": "direct",   "device": "mobile",  "date": "2025-11-08"},
    {"ts": pd.Timestamp("2025-11-08T10:07:00Z"), "user_id": "u1", "path": "/checkout",   "referrer": "direct",   "device": "mobile",  "date": "2025-11-08"},

    # user u2 navega pero no compra
    {"ts": pd.Timestamp("2025-11-08T11:00:00Z"), "user_id": "u2", "path": "/",          "referrer": "google",   "device": "desktop", "date": "2025-11-08"},
    {"ts": pd.Timestamp("2025-11-08T11:03:00Z"), "user_id": "u2", "path": "/productos",  "referrer": "google",   "device": "desktop", "date": "2025-11-08"},

    # user u3 dos sesiones por timeout (>30 min)
    {"ts": pd.Timestamp("2025-11-08T09:00:00Z"), "user_id": "u3", "path": "/",          "referrer": "facebook", "device": "tablet",  "date": "2025-11-08"},
    {"ts": pd.Timestamp("2025-11-08T09:10:00Z"), "user_id": "u3", "path": "/productos",  "referrer": "facebook", "device": "tablet",  "date": "2025-11-08"},
    {"ts": pd.Timestamp("2025-11-08T10:00:01Z"), "user_id": "u3", "path": "/",          "referrer": "facebook", "device": "tablet",  "date": "2025-11-08"},  # nueva sesión
])

events_oro, sessions, users_stats, top_paths, device_usage, sessions_per_day, funnel = build_gold(silver, session_timeout_min=30)

print("events_oro:", len(events_oro))
print("sessions:", len(sessions))
print("users_stats:", len(users_stats))
top_paths, device_usage, sessions_per_day, funnel


## 💾 Guardado de ORO y reporte

In [ ]:

# Guardado de tablas clave (como ejemplo)
gold_dir = BASE_DIR / 'gold_demo'
gold_dir.mkdir(parents=True, exist_ok=True)

events_oro_path = gold_dir / 'events_oro.parquet'
sessions_path   = gold_dir / 'sessions.parquet'
top_paths_path  = gold_dir / 'top_paths.parquet'

events_oro.to_parquet(events_oro_path, index=False)
sessions.to_parquet(sessions_path, index=False)
top_paths.to_parquet(top_paths_path, index=False)

# Construcción del reporte Markdown
args = Namespace(day='2025-11-08', gold=str(gold_dir), silver=str(gold_dir), report=str(gold_dir))
bronze_df = silver  # en la demo no cargamos BRONCE real
bad_df = pd.DataFrame()  # sin líneas rotas en esta demo

report_md = build_report_md(args, bronze_df, bad_df, silver, sessions, users_stats, top_paths, device_usage, sessions_per_day, funnel)

report_path = gold_dir / f"{args.day}-reporte.md"
report_path.write_text(report_md, encoding='utf-8')

report_path, events_oro_path, sessions_path, top_paths_path



### 📂 Salidas generadas
- `gold_demo/events_oro.parquet` — eventos con `session_id` y enriquecimientos.
- `gold_demo/sessions.parquet` — tabla de sesiones agregada con métricas y flags del embudo.
- `gold_demo/top_paths.parquet` — top de páginas.
- `gold_demo/2025-11-08-reporte.md` — reporte Markdown final.
